In [49]:
import numpy as np
import pandas as pd
import scipy as sp
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold, StratifiedKFold, train_test_split
from sklearn.metrics import mean_squared_error, log_loss
import sklearn
import os
import matplotlib.pyplot as plt
from matplotlib.pyplot import hist
from scipy import stats

In [50]:
# set random seed for numpy
RANDOM_SEED=100
np.random.seed(RANDOM_SEED)

In [51]:
# ATT and ATE AIPTW
def att_aiptw(Q0, Q1, g, A, Y, prob_t=None):
    """
    Double ML estimator for the ATT
    This uses the ATT specific scores, see equation 3.9 of https://www.econstor.eu/bitstream/10419/149795/1/869216953.pdf
    Return: aiptw of ATE and its standard error
    """
    
    # number of observations
    n = Y.shape[0]
    
    # estimate marginal probability of treatment
    if prob_t is None:
        prob_t = A.mean() 
    
    # att aiptw
    tau_hat = (A*(Y-Q0) - (1-A)*(g/(1-g))*(Y-Q0)).mean()/ prob_t
  
    # influence curve and standard error of aiptw
    phi = (A*(Y-Q0) - (1-A)*(g/(1-g))*(Y-Q0) - tau_hat*A) / prob_t
    std_hat = np.std(phi) / np.sqrt(n)

    return tau_hat, std_hat

def ate_aiptw(Q0, Q1, g, A, Y, prob_t=None):
    """
    Double ML estimator for the ATE
    Return: aiptw of ATE and its standard error
    """
    # number of observations
    n = Y.shape[0]
    
    # ate aiptw
    tau_hat = (Q1 - Q0 + A*(Y-Q1)/g - (1-A)*(Y-Q0)/(1-g)).mean()
  
    # influence curve and standard error of aiptw
    phi = Q1 - Q0 + A*(Y-Q1)/g - (1-A)*(Y-Q0)/(1-g) - tau_hat   
    std_hat = np.std(phi) / np.sqrt(n)

    return tau_hat, std_hat

In [52]:
# Conditional outcome models (Q models)
def make_linear_Q_model():
    ''' A function that returns a linear q model for later use in k-folding'''
    return LinearRegression()

def make_Q_model(output_type:str):
    ''' A function that returns a general ML q model for later use in k-folding'''
    if output_type == 'binary':
        return RandomForestClassifier(random_state=RANDOM_SEED, n_estimators=500, max_depth=None)
    return RandomForestRegressor(random_state=RANDOM_SEED, n_estimators=500, max_depth=None)
# One example: RandomForestRegressor(random_state=RANDOM_SEED, n_estimators=500, max_depth=None)

In [53]:
# Propensity score models (g models)
def make_g_model():
    ''' A function that returns a g model for computing propensity scores'''
    return RandomForestClassifier(n_estimators=100, max_depth=5)
# One example: RandomForestClassifier(n_estimators=100, max_depth=5)

In [54]:
# Functions for K-fold cross-fitting
def treatment_k_fold_fit_and_predict(make_model, X:pd.DataFrame, A:np.array, n_splits:int):
    '''
    Implements K fold cross-fitting for the model predicting the treatment A. 
    That is, 
    1. Split data into K folds
    2. For each fold j, the model is fit on the other K-1 folds
    3. The fitted model is used to make predictions for each data point in fold j
    Returns an array containing the predictions  

    Args:
    model: function that returns sklearn model (which implements fit and predict_prob)
    X: dataframe of variables to adjust for
    A: array of treatments
    n_splits: number of splits to use
    '''

    predictions = np.full_like(A, np.nan, dtype=float)
    kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=RANDOM_SEED)
    
    for train_index, test_index in kf.split(X, A):
        X_train = X.loc[train_index]
        A_train = A.loc[train_index]
        g = make_model()
        g.fit(X_train, A_train)

        # get predictions for split
        predictions[test_index] = g.predict_proba(X.loc[test_index])[:, 1]
    
    # sanity check that overlap holds
    assert np.isnan(predictions).sum() == 0
    return predictions

def outcome_k_fold_fit_and_predict(make_model, X:pd.DataFrame, y:np.array, A:np.array, n_splits:int, output_type:str):
    '''
    Implements K fold cross-fitting for the model predicting the outcome Y. 
    That is, 
    1. Split data into K folds
    2. For each fold j, the model is fit on the other K-1 folds
    3. The fitted model is used to make predictions for each data point in fold j
    Returns two arrays containing the predictions for all units untreated, all units treated  

    Args:
    model: function that returns sklearn model (that implements fit and either predict_prob or predict)
    X: dataframe of variables to adjust for
    y: array of outcomes
    A: array of treatments
    n_splits: number of splits to use
    output_type: type of outcome, "binary" or "continuous"
    '''

    predictions0 = np.full_like(A, np.nan, dtype=float)
    predictions1 = np.full_like(y, np.nan, dtype=float)
    if output_type == 'binary':
        kf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=RANDOM_SEED)
    elif output_type == 'continuous':
        kf = KFold(n_splits=n_splits, shuffle=True, random_state=RANDOM_SEED)

    # include the treatment as input feature
    X_w_treatment = X.copy()
    X_w_treatment["A"] = A

    # for predicting effect under treatment / control status for each data point 
    X0 = X_w_treatment.copy()
    X0["A"] = 0
    X1 = X_w_treatment.copy()
    X1["A"] = 1

    
    for train_index, test_index in kf.split(X_w_treatment, y):
        X_train = X_w_treatment.loc[train_index]
        y_train = y.loc[train_index]
        q = make_model(output_type)
        q.fit(X_train, y_train)

        if output_type =='binary':
            predictions0[test_index] = q.predict_proba(X0.loc[test_index])[:, 1]
            predictions1[test_index] = q.predict_proba(X1.loc[test_index])[:, 1]
        elif output_type == 'continuous':
            predictions0[test_index] = q.predict(X0.loc[test_index])
            predictions1[test_index] = q.predict(X1.loc[test_index])

    assert np.isnan(predictions0).sum() == 0
    assert np.isnan(predictions1).sum() == 0
    return predictions0, predictions1

In [55]:
def fit_and_run_model(df, outcome:str, treatment:str, confounders:list, make_g_model,
                      make_Q_model, n_splits=5, output_type='binary', ate=True):
    '''
    Function that creates a g, q, and aiptw model based on the 
    given inputs
    
    Inputs: df (pandas df) - the dataframe the variables are contained in
            outcome (str) - the outcome variable
            treatment (str) - the treatment variable
            confounders (lst) - a list of the confounding variables
            make_g_model - the make_g_model function
            make_Q_model - the make_Q_model function
            n_splits (int) - number of splits for the model
            output_type (str) - the desired output type, either binary or continous
            ate (bool) - whether to use ate or alternative att
    
    Returns: tau_hat - the tau hat estimator for the average treatment effect
             std of tau_hat - the standard deviation for the tau_hat estimator
    '''
    df = df.replace({outcome: .00001}, 0)
    df = df[[outcome] + confounders + [treatment]]
    df = df.dropna().reset_index()
    print('Running models for treatment {} and outcome {} on {} samples'.format(treatment, outcome, len(df)))

    outcome = df[outcome]
    confounders = df[confounders]
    treatment = df[treatment]
    treatment = treatment.replace({0.0: 0, 1.0: 1})
    outcome = outcome.replace({0.0: 0, 1.0: 1})
    g = treatment_k_fold_fit_and_predict(make_g_model, X=confounders, A=treatment, n_splits=n_splits)
    drop=False
    if min(g) < .01:
        print('\nWARNING:\n Some propensity scores are very small,\n which could '
              'lead to an inflated AIPTW.\n Minimum score = ', min(g))
    if max(g) > .99:
        print('\nWARNING:\n Some propensity scores are very large,\n which could '
              'lead to an inflated AIPTW.\n Maximum score = ', max(g))
        drop = True
    print('G Model has been fit')

    Q0_ml, Q1_ml = outcome_k_fold_fit_and_predict(make_Q_model, X=confounders, y=outcome, A=treatment, \
                                                  n_splits=n_splits, output_type=output_type)
    
    print('Q model has been fit')
    data_and_nuisance_estimates_ml = pd.DataFrame({'g': g, 'Q0': Q0_ml, 'Q1': Q1_ml, 'A': treatment, 'Y': outcome})
    
    if drop:
        data_and_nuisance_estimates_ml = data_and_nuisance_estimates_ml[data_and_nuisance_estimates_ml['g'] > .01]
        data_and_nuisance_estimates_ml = data_and_nuisance_estimates_ml[data_and_nuisance_estimates_ml['g'] < .99]
        print('Dropped {} observations due to overlap condition'.format(len(df) - len(data_and_nuisance_estimates_ml)))
    n = len(data_and_nuisance_estimates_ml)
    # ate aiptw
    if ate:
        tau_hat, std_hat = ate_aiptw(**data_and_nuisance_estimates_ml)
    else: 
        tau_hat, std_hat = att_aiptw(**data_and_nuisance_estimates_ml)
    test_stat = tau_hat / std_hat
    p_value = stats.t.sf(abs(test_stat), df=(n-1))
    print('AIPTW model has been fit. Returning \u03C4 hat and its standard error')
    print('\u03C4 hat = {}, std = {}, test statistic = {}, p-value = {}\n'.format(round(tau_hat, 5), \
                                                                                  round(std_hat, 5), \
                                                                                  round(test_stat, 5), \
                                                                                  round(p_value, 5)))
    return tau_hat, std_hat, test_stat, p_value

In [56]:
faculty_df = pd.read_csv("../data/final_df.csv")

In [57]:
faculty_df.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'lname', 'fname', 'key', 'rank', 'status',
       'tenure', 'emeritus', 'year', 'nationality', 'university', 'psych',
       'esfellow', 'indian', 'otherasian', 'jewish', 'indian2', 'otherasian2',
       'jewish2', 'nobel', 'clark', 'first_letter', 'first_letter_fname',
       'letter_as_number', 'nobel_or_clark', 'letter_binarized',
       'avg_authorship', 'citations'],
      dtype='object')

In [58]:
outcome = "tenure"
treatment = "letter_binarized"
confounders = ["indian",
               "otherasian",
               "jewish",
               'indian2',
               'otherasian2',
               'jewish2',
               'citations']

In [59]:
df_dict = {'econ': [], 'ranking_cap': [], 
           'tau': [], 'tau_stderr': [], 
           'test_stat': [], 'p_value': []}
for psych in [0, 1]:
    for ranking_cap in [5, 10, 25, 35]:
        sample_df = faculty_df[faculty_df['psych'] == psych]
        tau, tau_stderr, test_stat, p_value = fit_and_run_model(sample_df[sample_df['rank'] <= ranking_cap], \
                                                                outcome, treatment, confounders, make_g_model, \
                                                                make_Q_model)
        
        econ = 1 - psych
        df_dict['econ'].append(econ)
        df_dict['ranking_cap'].append(ranking_cap)
        df_dict['tau'].append(tau)
        df_dict['tau_stderr'].append(tau_stderr)
        df_dict['test_stat'].append(test_stat)
        df_dict['p_value'].append(p_value)

results_df = pd.DataFrame(df_dict)

Running models for treatment letter_binarized and outcome tenure on 282 samples
G Model has been fit
Q model has been fit
AIPTW model has been fit. Returning τ hat and its standard error
τ hat = -0.11281, std = 0.04911, test statistic = -2.29691, p-value = 0.01118

Running models for treatment letter_binarized and outcome tenure on 530 samples
G Model has been fit
Q model has been fit
AIPTW model has been fit. Returning τ hat and its standard error
τ hat = -0.04386, std = 0.03502, test statistic = -1.25217, p-value = 0.10553

Running models for treatment letter_binarized and outcome tenure on 1193 samples
G Model has been fit
Q model has been fit
AIPTW model has been fit. Returning τ hat and its standard error
τ hat = 0.01321, std = 0.0251, test statistic = 0.52655, p-value = 0.2993

Running models for treatment letter_binarized and outcome tenure on 1513 samples
G Model has been fit
Q model has been fit
AIPTW model has been fit. Returning τ hat and its standard error
τ hat = 0.01632, 

In [60]:
results_df

,econ,ranking_cap,tau,tau_stderr,test_stat,p_value
0,1,5,-0.112810,0.049114,-2.296907,0.011180
1,1,10,-0.043857,0.035025,-1.252174,0.105530
2,1,25,0.013214,0.025095,0.526546,0.299303
3,1,35,0.016316,0.021474,0.759821,0.223740
4,0,5,-0.004404,0.037309,-0.118039,0.453045
5,0,10,-0.003424,0.032549,-0.105199,0.458125
6,0,25,-0.030688,0.021967,-1.396967,0.081330
7,0,35,-0.013897,0.018118,-0.766997,0.221593


In [17]:
def color_significant(v, color1, color2):
    if v < .01:
        return f"background-color: {color1};"
    elif v < .05:
        return f"background-color: {color2};"
    return None

In [61]:
results_df.style.applymap(color_significant, color1='salmon', color2='yellow',
                          subset=['p_value'])

,econ,ranking_cap,tau,tau_stderr,test_stat,p_value
0,1,5,-0.112810,0.049114,-2.296907,0.011180
1,1,10,-0.043857,0.035025,-1.252174,0.105530
2,1,25,0.013214,0.025095,0.526546,0.299303
3,1,35,0.016316,0.021474,0.759821,0.223740
4,0,5,-0.004404,0.037309,-0.118039,0.453045
5,0,10,-0.003424,0.032549,-0.105199,0.458125
6,0,25,-0.030688,0.021967,-1.396967,0.081330
7,0,35,-0.013897,0.018118,-0.766997,0.221593


In [62]:
import statsmodels.formula.api as smf

In [63]:
d = {'econ': [], 'ranking_cap': [], 
           'coef': [], 'std_err': [], 
           'test_stat': [], 'p_value': []}
reg_str = 'tenure ~ letter_as_number + indian + otherasian + jewish +' \
          + 'indian2 + otherasian2 + jewish2 + citations'
for psych in [0,1]:
    for ranking_cap in [5, 10, 25, 35]:
        mod = smf.logit(reg_str, \
                data=faculty_df[(faculty_df['psych'] == psych) & (faculty_df['rank'] <= ranking_cap)]).fit()
        
        d['econ'].append(1 - psych)
        d['ranking_cap'].append(ranking_cap)
        d['coef'].append(mod.params[1])
        d['std_err'].append(mod.bse[1])
        d['test_stat'].append(mod.tvalues[1])
        d['p_value'].append(mod.pvalues[1])

results_df_logit_asnum = pd.DataFrame(d)

Optimization terminated successfully.
         Current function value: 0.470531
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.480433
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.501604
         Iterations 7
Optimization terminated successfully.
         Current function value: 0.498718
         Iterations 7
         Current function value: 0.431006
         Iterations: 35
         Current function value: 0.449278
         Iterations: 35
         Current function value: 0.429218
         Iterations: 35
         Current function value: 0.421499
         Iterations: 35


/Users/mariatamayo/miniconda3/envs/ML-Econ/lib/python3.10/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/mariatamayo/miniconda3/envs/ML-Econ/lib/python3.10/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/mariatamayo/miniconda3/envs/ML-Econ/lib/python3.10/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/mariatamayo/miniconda3/envs/ML-Econ/lib/python3.10/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimizat

In [64]:
results_df_logit_asnum

,econ,ranking_cap,coef,std_err,test_stat,p_value
0,1,5,-0.046695,0.023440,-1.992114,0.046359
1,1,10,-0.036473,0.017079,-2.135513,0.032719
2,1,25,-0.002578,0.010684,-0.241325,0.809303
3,1,35,-0.006337,0.009505,-0.666750,0.504932
4,0,5,0.013609,0.019228,0.707771,0.479088
5,0,10,0.013543,0.015850,0.854437,0.392863
6,0,25,-0.003351,0.011401,-0.293962,0.768787
7,0,35,-0.003512,0.009862,-0.356053,0.721801


In [65]:
results_df_logit_asnum.style.applymap(color_significant, color1='salmon', color2='yellow',
                          subset=['p_value'])

,econ,ranking_cap,coef,std_err,test_stat,p_value
0,1,5,-0.046695,0.023440,-1.992114,0.046359
1,1,10,-0.036473,0.017079,-2.135513,0.032719
2,1,25,-0.002578,0.010684,-0.241325,0.809303
3,1,35,-0.006337,0.009505,-0.666750,0.504932
4,0,5,0.013609,0.019228,0.707771,0.479088
5,0,10,0.013543,0.015850,0.854437,0.392863
6,0,25,-0.003351,0.011401,-0.293962,0.768787
7,0,35,-0.003512,0.009862,-0.356053,0.721801


In [66]:
d = {'econ': [], 'ranking_cap': [], 
           'coef': [], 'std_err': [], 
           'test_stat': [], 'p_value': []}
reg_str = 'tenure ~ letter_binarized + indian + otherasian + jewish +' \
          + '+ citations'
for psych in [0,1]:
    for ranking_cap in [5, 10, 25, 35]:
        print(psych, ranking_cap)
        mod = smf.logit(reg_str, \
                data=faculty_df[(faculty_df['psych'] == psych) & (faculty_df['rank'] <= ranking_cap)]).fit()
        
        d['econ'].append(1 - psych)
        d['ranking_cap'].append(ranking_cap)
        d['coef'].append(mod.params[1])
        d['std_err'].append(mod.bse[1])
        d['test_stat'].append(mod.tvalues[1])
        d['p_value'].append(mod.pvalues[1])

results_df_logit_bin = pd.DataFrame(d)

0 5
Optimization terminated successfully.
         Current function value: 0.478817
         Iterations 7
0 10
Optimization terminated successfully.
         Current function value: 0.483179
         Iterations 7
0 25
Optimization terminated successfully.
         Current function value: 0.502399
         Iterations 7
0 35
Optimization terminated successfully.
         Current function value: 0.499845
         Iterations 7
1 5
         Current function value: 0.436621
         Iterations: 35
1 10
Optimization terminated successfully.
         Current function value: 0.452868
         Iterations 6
1 25
Optimization terminated successfully.
         Current function value: 0.431248
         Iterations 6
1 35
Optimization terminated successfully.
         Current function value: 0.423439
         Iterations 6


/Users/mariatamayo/miniconda3/envs/ML-Econ/lib/python3.10/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [67]:
results_df_logit_bin.style.applymap(color_significant, color1='salmon', color2='yellow',
                          subset=['p_value'])

,econ,ranking_cap,coef,std_err,test_stat,p_value
0,1,5,-0.537157,0.308641,-1.740393,0.081790
1,1,10,-0.385287,0.223893,-1.720852,0.085278
2,1,25,-0.102820,0.143529,-0.716373,0.473761
3,1,35,-0.101947,0.127846,-0.797421,0.425207
4,0,5,0.099190,0.258227,0.384120,0.700890
5,0,10,0.069628,0.211482,0.329238,0.741976
6,0,25,-0.141170,0.152844,-0.923621,0.355684
7,0,35,-0.125066,0.133648,-0.935791,0.349381
